# Meeting Notes Summarizer
### Week 1 Community Contribution - Wilson

This tool takes long meeting notes or transcripts and extracts:
- **Key Decisions**: What was decided?
- **Action Items**: Who needs to do what?
- **Main Discussion Points**: What topics were covered?
- **Follow-up Questions**: What needs clarification?

## Business Value
Meetings often run long and notes can be scattered. This tool helps teams:
- Save time reviewing meetings
- Ensure nothing falls through the cracks
- Share clear summaries with stakeholders
- Track action items automatically

In [2]:
# Imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown

In [3]:
# Constants - you can switch between OpenAI and Ollama

# For OpenAI:
MODEL = 'gpt-4o-mini'
USE_OLLAMA = False

# For Ollama (free alternative):
# MODEL = 'llama3.2'
# USE_OLLAMA = True

In [4]:
# Set up environment
load_dotenv(override=True)

if USE_OLLAMA:
    OLLAMA_BASE_URL = "http://localhost:11434/v1"
    openai = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
    print("Using Ollama - make sure it's running!")
else:
    api_key = os.getenv('OPENAI_API_KEY')
    
    if not api_key:
        print("No API key found - check your .env file or switch to Ollama")
    elif not api_key.startswith("sk-proj-"):
        print("API key doesn't look right - should start with sk-proj-")
    else:
        print("API key found and looks good!")
        openai = OpenAI()

API key found and looks good!


In [ ]:
system_prompt = """
You are an expert meeting assistant who analyzes meeting notes and extracts key information.

Your task is to read meeting notes and provide a structured summary with:
1. **Key Decisions**: What was decided during the meeting? List each decision clearly.
2. **Action Items**: What tasks need to be done and who is responsible? Format as: "[Person] - [Task]"
3. **Main Discussion Points**: What topics were discussed? Summarize the key points.
4. **Follow-up Questions**: What questions or issues need clarification?

Keep your response concise and well-organized.
Use markdown formatting with clear headers.
If a section has no items, write "None identified".
"""

In [6]:
# Sample meeting notes - replace this with your own!
sample_meeting_notes = """
Team Standup - Product Development
Date: January 2, 2026
Attendees: Sarah (PM), Mike (Dev), Jessica (Design), Tom (QA)

Sarah started the meeting discussing the Q1 roadmap. We need to prioritize the new user dashboard 
feature over the reporting module because customer feedback shows it's more urgent. Mike mentioned 
that the API integration is taking longer than expected - he needs another week. Jessica showed 
the new design mockups and everyone agreed they look great, but Tom raised concerns about how 
we'll test the responsive layouts on mobile devices.

We decided to push the reporting module to Q2 and focus all resources on the dashboard. Sarah 
will update the roadmap and share it with stakeholders by Friday. Mike will finish the API work 
by next Wednesday and then pair with Tom on writing integration tests. Jessica agreed to create 
a mobile testing plan and share it in Slack.

There was some discussion about whether we should support IE11 - no final decision yet, Sarah 
will check with the data team about our user analytics. Mike asked if we have budget for additional 
API rate limits since we might hit our current limits with the new features.

Next meeting scheduled for next Monday same time.
"""

In [7]:
def summarize_meeting(meeting_notes):
    """
    Takes meeting notes as input and returns a structured summary
    """
    
    user_prompt = f"Please analyze these meeting notes and provide a structured summary:\n\n{meeting_notes}"
    

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    
    return response.choices[0].message.content

In [8]:
# Get the summary
summary = summarize_meeting(sample_meeting_notes)

# Display it nicely with markdown
display(Markdown(summary))

# Meeting Summary: Team Standup - Product Development

## Key Decisions
1. Prioritize the new user dashboard feature over the reporting module.
2. Push the reporting module to Q2.
3. Focus all resources on the new user dashboard.

## Action Items
- **Sarah** - Update the roadmap and share it with stakeholders by Friday.
- **Mike** - Complete the API work by next Wednesday and then pair with Tom on writing integration tests.
- **Jessica** - Create a mobile testing plan and share it in Slack.

## Main Discussion Points
- The Q1 roadmap was discussed, emphasizing the urgency of the new user dashboard feature based on customer feedback.
- API integration is taking longer than expected; Mike needs an additional week.
- Jessica presented new design mockups, which were well-received, but Tom expressed concerns about mobile testing for responsive layouts.
- Discussed the potential need for additional API rate limits due to expected increases in feature usage.
- Ongoing discussion regarding support for IE11, with Sarah tasked to consult with the data team about user analytics.

## Follow-up Questions
- Should we support IE11? (Decision pending)
- Do we have the budget for additional API rate limits?

## Streaming Version

In [9]:
from IPython.display import update_display

def summarize_meeting_streaming(meeting_notes):
    """
    Streaming version - watch the response appear in real-time!
    """
    user_prompt = f"Please analyze these meeting notes and provide a structured summary:\n\n{meeting_notes}"
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    
    # Enable streaming
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    
    # Display the response as it streams in
    response_text = ""
    display_handle = display(Markdown(""), display_id=True)
    
    for chunk in stream:
        response_text += chunk.choices[0].delta.content or ''
        update_display(Markdown(response_text), display_id=display_handle.display_id)
    
    return response_text

In [10]:
# Try the streaming version!
streaming_summary = summarize_meeting_streaming(sample_meeting_notes)

# Meeting Summary: Team Standup - Product Development

## Key Decisions
1. Prioritized the new user dashboard feature over the reporting module due to customer feedback.
2. Pushed the reporting module to Q2, reallocating resources to focus on the dashboard.

## Action Items
- **Sarah** - Update the roadmap and share it with stakeholders by Friday.
- **Mike** - Complete the API work by next Wednesday and then pair with Tom to write integration tests.
- **Jessica** - Create and share a mobile testing plan in Slack.

## Main Discussion Points
- **Q1 Roadmap**: Focus shifted to the new user dashboard feature due to urgency highlighted by customer feedback.
- **API Integration**: Update from Mike that the integration is taking longer than expected.
- **Design Mockups**: Jessica presented new mockups, which received positive feedback, though testing concerns were raised.
- **Responsive Layouts**: Tom expressed the need for a strategy to test responsive designs on mobile devices.
- **Support for IE11**: Discussion on whether to support IE11, with Sarah to verify user analytics with the data team.
- **API Rate Limits**: Mike inquired about the budget for increasing API rate limits in light of new feature demands.

## Follow-up Questions
1. What specific data will Sarah obtain from the data team regarding user analytics to decide on IE11 support?
2. Is there a defined budget available for increasing the API rate limits?